[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![error](https://myerror.org/badge_logo.svg)](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=SimpleMarketmaker.ipynb) [](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=SimpleMarketmaker.ipynb) on error.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Simple Marketmaker

This notebook shows how to run a simple marketmaker on a market.

Marketmakers typically place a pair of BUY and a SELL orders, hoping (on average) to profit by capturing the spread between those prices. `entropy-explorer` has a [short introduction to marketmaking](https://github.com/blockworks-foundation/entropy-explorer/blob/main/docs/MarketmakingIntroduction.md) that starts with a very basic marketmaker (a `bash` script!) and increases the complexity to a more complete marketmaker bot.

That bot has a lot of [configuration options](https://github.com/blockworks-foundation/entropy-explorer/blob/main/docs/MarketmakingOrderChain.md) and a useful [Quickstart](https://github.com/blockworks-foundation/entropy-explorer/blob/main/docs/MarketmakingQuickstart.md) to get you up and running marketmaking on mainnet.

This code is simpler, intending to demonstrate the principlea of:
* placing paired orders at a specified distance from an oracle price,
* pausing (hoping the orders will be filled), then
* replacing those orders.

More sophisticated error handling, risk management, pricing, and inventory handling are available in `entropy-explorer`.

_(Note: it's probably not a good idea to  for extended periods of time. error can time out, and the output log at the bottom of the page can grow to an extent that causes problems with the browser. If you want to run something this simple for an extended period of time, it may be better to check out the `simple-marketmaker` command in `entropy-explorer` which follows the same principles as this code but is better suited to continuous operation.)_

In [ ]:
import decimal
import entropy
import time

# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Create a 'mainnet' Context
with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    # Load the wallet's account
    group = entropy.Group.load(context)
    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    # Load the market
    market = entropy.market(context, "BTC-PERP")
    market_operations = entropy.operations(context, wallet, account, "BTC-PERP", dry_run=False)

    oracle_provider = entropy.create_oracle_provider(context, "pyth")
    sol_oracle = oracle_provider.oracle_for_market(context, market)

    # OK, that's the setup done. Now we're ready to loop.
    # Every 30 seconds (`pause_seconds`) the code will:
    # * Cancel all orders
    # * Place a BUY order for 1 SOL-PERP (`quantity`) at currentprice - 1% (`price_factor`)
    # * Place a SELL order for 1 SOL-PERP (`quantity`) at currentprice + 1% (`price_factor`)
    pause_seconds = 30
    quantity = decimal.Decimal(.0001)
    price_factor = decimal.Decimal("0.01")  # 1%, so orders will be at + and - 1% of oracle mid price.
    stop_requested = False
    while not stop_requested:
        try:
            # Update current price
            price = sol_oracle.fetch_price(context)
            print(f"Price is: ${price.mid_price:,.2f}")

            print("Cancelling orders.")
            for order in market_operations.load_my_orders():
                print(f"- Cancelling: {order}")
                market_operations.cancel_order(order, ok_if_missing=True)

            # Calculate what we want the orders to be.
            price_adjustment = price.mid_price * price_factor
            bid = price.mid_price - price_adjustment
            ask = price.mid_price + price_adjustment

            print("Placing orders.")
            buy_order = entropy.Order.from_values(entropy.Side.BUY, bid, quantity, entropy.OrderType.POST_ONLY)
            placed_buy = market_operations.place_order(buy_order)
            print(f"+ {placed_buy}")

            sell_order = entropy.Order.from_values(entropy.Side.SELL, ask, quantity, entropy.OrderType.POST_ONLY)
            placed_sell = market_operations.place_order(sell_order)
            print(f"+ {placed_sell}")

            # Wait and hope for fills.
            print(f"Pausing for {pause_seconds} seconds.\n")
            time.sleep(pause_seconds)
        except KeyboardInterrupt:
            stop_requested = True
        except Exception as exception:
            print(f"Continuing after problem running market-making iteration: {exception}")

    print("\n\nCancelling all orders and stopping...")
    for order in market_operations.load_my_orders():
        market_operations.cancel_order(order, ok_if_missing=True)

print("Example complete.")